## Classifier analysis part 1 (Supp. Figure 3)

This notebook applies augur to calculate AUC scores for each cell type in each condition. Because the version of Augur that we used is not compatible with the version of Seurat primarily used throughout the project, this notebook calculates AUC scores on seurat objects that are transformed to a previous version of Seurat. Part 2 will then detail how we performed analysis on these values.

#### Import libraries (run during both R versions of the code)

In [1]:
library(Seurat)
library(dplyr)
library(magrittr)
library(ggplot2)
library(xlsx)
library(tidyr)
library(scales)
library(reshape2)
library(SeuratObject)

Attaching SeuratObject


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'tidyr'


The following object is masked from 'package:magrittr':

    extract



Attaching package: 'reshape2'


The following object is masked from 'package:tidyr':

    smiths




#### Loading RDS files

Run as R version 4.1.2. Need to convert Seurat object to an older version via SCE

In [12]:
LS.integrated <- readRDS('C:/Users/stuberadmin/Documents/scRNAseq/211119-RS-rv4/Neurons_only_iter2/LS_integrated.rds')
LS_sal<-readRDS(file = "C:/Users/stuberadmin/Documents/scRNAseq/211119-RS/Neurons_only_iter5/LS_sal.rds")
LS_mor<-readRDS(file = "C:/Users/stuberadmin/Documents/scRNAseq/211119-RS/Neurons_only_iter5/LS_mor.rds")
LS_nal<-readRDS(file = "C:/Users/stuberadmin/Documents/scRNAseq/211119-RS/Neurons_only_iter5/LS_nal.rds")
LS_one_mor<-readRDS( file = "C:/Users/stuberadmin/Documents/scRNAseq/211119-RS/Neurons_only_iter5/LS_one_mor.rds")
LS_nal_no_mor<-readRDS( file = "C:/Users/stuberadmin/Documents/scRNAseq/211119-RS/Neurons_only_iter5/LS_nal_no_mor.rds")

In [7]:
BiocManager::install("SingleCellExperiment") #to convert to SCE

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.2 (2021-11-01)

Warning message:
"package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'SingleCellExperiment'"
Installation paths not writeable, unable to update packages
  path: C:/Program Files/R/R-4.1.2/library
  packages:
    boot, class, cluster, codetools, foreign, KernSmooth, lattice, MASS, mgcv,
    nlme, nnet, rpart, spatial, survival

Old packages: 'ape', 'bayestestR', 'BH', 'BiocManager', 'bit', 'blob', 'broom',
  'bslib', 'cachem', 'car', 'caret', 'checkmate', 'classInt', 'clipr',
  'clustree', 'colorspace', 'commonmark', 'conquer', 'cpp11', 'crayon', 'curl',
  'data.table', 'datawizard', 'DBI', 'dbplyr', 'deldir', 'dendextend',
  'DEoptimR', 'digest', 'dplyr', 'DT', 'dtplyr', 'e1071', 'effectsize',
  'ellipse', 'euler

In [16]:
LS.integrated_SCE <- as.SingleCellExperiment(LS.integrated, assay='RNA')
LS_sal_SCE <- as.SingleCellExperiment(LS_sal, assay='RNA')
LS_mor_SCE <- as.SingleCellExperiment(LS_mor, assay='RNA')
LS_nal_SCE <- as.SingleCellExperiment(LS_nal, assay='RNA')
LS_one_mor_SCE <- as.SingleCellExperiment(LS_one_mor, assay='RNA')
LS_nal_no_mor_SCE <- as.SingleCellExperiment(LS_nal_no_mor, assay='RNA')

In [17]:
saveRDS(LS.integrated_SCE, 'LS_integrated_SCE.rds')
saveRDS(LS_sal_SCE, 'LS_sal_SCE.rds')
saveRDS(LS_mor_SCE, 'LS_mor_SCE.rds')
saveRDS(LS_nal_SCE, 'LS_nal_SCE.rds')
saveRDS(LS_one_mor_SCE, 'LS_one_mor_SCE.rds')
saveRDS(LS_nal_no_mor_SCE, 'LS_nal_no_mor_SCE.rds')

#### Now run as R version 3.6.3 

Load in SCE objects

In [18]:
LS.integrated_SCE <- readRDS('LS_integrated_SCE.rds')
LS_sal_SCE <- readRDS('LS_sal_SCE.rds')
LS_mor_SCE <- readRDS('LS_mor_SCE.rds')
LS_nal_SCE <- readRDS('LS_nal_SCE.rds')
LS_one_mor_SCE <- readRDS('LS_one_mor_SCE.rds')
LS_nal_no_mor_SCE <- readRDS('LS_nal_no_mor_SCE.rds')

Now reconvert to old version of seurat objects that we can use for the classifier analysis

In [19]:
LS.integrated<-as.Seurat(LS.integrated_SCE)
LS_sal <- as.Seurat(LS_sal_SCE)
LS_mor <- as.Seurat(LS_mor_SCE)
LS_nal <- as.Seurat(LS_nal_SCE)
LS_one_mor <- as.Seurat(LS_one_mor_SCE)
LS_nal_no_mor <- as.Seurat(LS_nal_no_mor_SCE)

In [20]:
Idents(LS.integrated)

AAACGCTCAGGCACAALS_sal        AAAGAACCAGCAGATGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAAGAACGTCCGAAAGLS_sal        AAAGGATGTGTTTCTTLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AACAAAGGTTTGTTCTLS_sal        AACACACCAGAGAAAGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AACAGGGCATCGAACTLS_sal        AACCACAAGCAGCCTCLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AACCACATCAGGACAGLS_sal        AACCCAACATGAATAGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AACGAAAAGAAGATCTLS_sal        AACGGGATCAGTAGGGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGAACATCATGCATGLS_sal        AAGACAAGTCACAGAGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGCATCGTTTCCAAGLS_sal        AAGCGAGCAGCTGGTCLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGCGTTAGCTACAAALS_sal        AAGCGTTGTGTGTGGALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGCGTTTCCAACTGALS_sal        AAGGAATTCTTAAGGCLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGTCGTAGAACCCGALS_sal        AAGTGAAAGCAATAACLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGTGAACATTAAGCCLS_sal        AAGTGAATCGACTCCTLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AAGTTCGTCACATACGLS_sal        AATAGAGCAGCTGTATLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AATAGAGTCTCAGTCCLS_sal        AATCGACAGGTAATCALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AATCGACCATCGGAGALS_sal        AATCGTGTCAAGCCTALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AATGCCATCTGAGATCLS_sal        AATGGAAAGCTACAAALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AATGGAAGTGCAACAGLS_sal        AATGGCTAGTGGTGACLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       AATTCCTTCATGAGAALS_sal        AATTCCTTCGCTAATGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACAAAGAAGCTGAAATLS_sal        ACAAAGAAGGCCACTCLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACAAGCTCAATACAGALS_sal        ACAAGCTCATGTTCAGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACACCAACACAGCGCTLS_sal        ACACGCGGTACTGAGGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACACTGAAGAGAGCCTLS_sal        ACAGAAAAGGTTCCGCLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACAGAAACATGGTGGALS_sal        ACAGAAAGTGCAACAGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACATCCCCAAATAAGCLS_sal        ACATGCAAGGGCCAATLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACATGCAAGTAAACACLS_sal        ACATTTCCAGTCGGAALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACCAAACGTAGTCACTLS_sal        ACCAAACGTCTCGGGTLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACCAACAGTCTTCAAGLS_sal        ACCAACAGTTAGCGGALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACCACAACAAGATGGCLS_sal        ACCACAAGTGATGAATLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACCATTTGTTCCGCGALS_sal        ACCCAAATCTGTGTGALS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACCCTCAAGGGCAACTLS_sal        ACCTGAACACAAAGCGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACGATCACATGCTGCGLS_sal        ACGATCAGTCAAACGGLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACGTCCTCAGAGGTACLS_sal        ACGTTCCAGGCTCTATLS_sal 
         SingleCellExperiment          SingleCellExperiment 
       ACGTTCCCATCGGCCALS_sal        ACTATCTGTGGTTCTALS_sal 
         SingleCellExperiment          Sin

Reassign true identities from the metadata

In [21]:
Idents(LS.integrated) <- LS.integrated@meta.data$ident

In [22]:
Idents(LS.integrated)

AAACGCTCAGGCACAALS_sal        AAAGAACCAGCAGATGLS_sal 
                            1                             1 
       AAAGAACGTCCGAAAGLS_sal        AAAGGATGTGTTTCTTLS_sal 
                            1                             1 
       AACAAAGGTTTGTTCTLS_sal        AACACACCAGAGAAAGLS_sal 
                            1                             1 
       AACAGGGCATCGAACTLS_sal        AACCACAAGCAGCCTCLS_sal 
                            1                             1 
       AACCACATCAGGACAGLS_sal        AACCCAACATGAATAGLS_sal 
                            1                             1 
       AACGAAAAGAAGATCTLS_sal        AACGGGATCAGTAGGGLS_sal 
                            1                             4 
       AAGAACATCATGCATGLS_sal        AAGACAAGTCACAGAGLS_sal 
                           10                             1 
       AAGCATCGTTTCCAAGLS_sal        AAGCGAGCAGCTGGTCLS_sal 
                            1                             1 
       AAGCGTTAGCTACAAALS_sal        AAGCGTTGTGTGTGGALS_sal 
                            1                             1 
       AAGCGTTTCCAACTGALS_sal        AAGGAATTCTTAAGGCLS_sal 
                            1                             1 
       AAGTCGTAGAACCCGALS_sal        AAGTGAAAGCAATAACLS_sal 
                            1                             1 
       AAGTGAACATTAAGCCLS_sal        AAGTGAATCGACTCCTLS_sal 
                            1                             1 
       AAGTTCGTCACATACGLS_sal        AATAGAGCAGCTGTATLS_sal 
                            1                             1 
       AATAGAGTCTCAGTCCLS_sal        AATCGACAGGTAATCALS_sal 
                            1                             1 
       AATCGACCATCGGAGALS_sal        AATCGTGTCAAGCCTALS_sal 
                            1                             1 
       AATGCCATCTGAGATCLS_sal        AATGGAAAGCTACAAALS_sal 
                            1                            10 
       AATGGAAGTGCAACAGLS_sal        AATGGCTAGTGGTGACLS_sal 
                            1                             1 
       AATTCCTTCATGAGAALS_sal        AATTCCTTCGCTAATGLS_sal 
                            1                             1 
       ACAAAGAAGCTGAAATLS_sal        ACAAAGAAGGCCACTCLS_sal 
                            1                             1 
       ACAAGCTCAATACAGALS_sal        ACAAGCTCATGTTCAGLS_sal 
                            1                             1 
       ACACCAACACAGCGCTLS_sal        ACACGCGGTACTGAGGLS_sal 
                           13                             1 
       ACACTGAAGAGAGCCTLS_sal        ACAGAAAAGGTTCCGCLS_sal 
                            4                             1 
       ACAGAAACATGGTGGALS_sal        ACAGAAAGTGCAACAGLS_sal 
                            1                             1 
       ACATCCCCAAATAAGCLS_sal        ACATGCAAGGGCCAATLS_sal 
                            1                             1 
       ACATGCAAGTAAACACLS_sal        ACATTTCCAGTCGGAALS_sal 
                            1                             1 
       ACCAAACGTAGTCACTLS_sal        ACCAAACGTCTCGGGTLS_sal 
                            1                             1 
       ACCAACAGTCTTCAAGLS_sal        ACCAACAGTTAGCGGALS_sal 
                            1                             1 
       ACCACAACAAGATGGCLS_sal        ACCACAAGTGATGAATLS_sal 
                            1                             1 
       ACCATTTGTTCCGCGALS_sal        ACCCAAATCTGTGTGALS_sal 
                            1                             1 
       ACCCTCAAGGGCAACTLS_sal        ACCTGAACACAAAGCGLS_sal 
                            1                             1 
       ACGATCACATGCTGCGLS_sal        ACGATCAGTCAAACGGLS_sal 
                            1                             1 
       ACGTCCTCAGAGGTACLS_sal        ACGTTCCAGGCTCTATLS_sal 
                            1                             1 
       ACGTTCCCATCGGCCALS_sal        ACTATCTGTGGTTCTALS_sal 
                            3             

#### Assigning cell identities

In [23]:
new.ident <- c("Gaba1","Gaba2","Gaba3","Gaba4","Gaba5","Gaba6","Gaba7","Glu1","Gaba8","Gaba9","Gaba10","Glu2","Gaba11","Gaba12")
names(x = new.ident) <- levels(x =LS.integrated)
LS.integrated<- RenameIdents(object =LS.integrated, new.ident)

In [24]:
table(Idents(LS.integrated))


 Gaba1  Gaba2  Gaba3  Gaba4  Gaba5  Gaba6  Gaba7   Glu1  Gaba8  Gaba9 Gaba10 
  3124   2731   2316   2248   2157   2116   2028   1919   1904   1860   1634 
  Glu2 Gaba11 Gaba12 
  1015    437    118 

In [25]:
for (i in 1:length(new.ident)){
assign(paste(new.ident[i],"_barcode",sep=""),colnames(LS.integrated@assays$RNA@data[,which(Idents(object=LS.integrated) %in% new.ident[i])]))# this gives all barcodes in cluster
assign(paste(new.ident[i],"_barcode_LS_sal",sep=""),intersect(colnames(LS_sal@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
assign(paste(new.ident[i],"_barcode_LS_mor",sep=""),intersect(colnames(LS_mor@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
assign(paste(new.ident[i],"_barcode_LS_nal",sep=""),intersect(colnames(LS_nal@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
assign(paste(new.ident[i],"_barcode_LS_one_mor",sep=""),intersect(colnames(LS_one_mor@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
assign(paste(new.ident[i],"_barcode_LS_nal_no_mor",sep=""),intersect(colnames(LS_nal_no_mor@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
}

In [26]:
celltype<-vector()
for (i in 1:dim(LS.integrated@meta.data)[1]){
    celltype[i]<-toString(new.ident[LS.integrated@meta.data$integrated_snn_res.0.2[i]])
}
LS.integrated@meta.data$cell_type<-celltype

In [27]:
LS.integrated <- NormalizeData(LS.integrated)

In [28]:
DefaultAssay(LS.integrated) <- 'RNA'

We performed comparisons between Sal and Mor, and Mor and Nal. To do this, we split up the seurat object by experimental group

In [29]:
LS_nonal <- subset(LS.integrated, cells = c(colnames(LS_sal),colnames(LS_mor)))
LS_nonal

An object of class Seurat 
26857 features across 10923 samples within 1 assay 
Active assay: RNA (26857 features, 0 variable features)
 2 dimensional reductions calculated: PCA, UMAP

In [30]:
LS_nosal <- subset(LS.integrated, cells = c(colnames(LS_mor),colnames(LS_nal)))
LS_nosal

An object of class Seurat 
26857 features across 11358 samples within 1 assay 
Active assay: RNA (26857 features, 0 variable features)
 2 dimensional reductions calculated: PCA, UMAP

#### Calculating condition- and cell-type based AUC scores

In [ ]:
library(Augur)

In [ ]:
celltype<-vector()
for (i in 1:dim(LS_nonal@meta.data)[1]){
    celltype[i]<-toString(new.ident[LS_nonal@meta.data$integrated_snn_res.0.2[i]])
}
LS_nonal@meta.data$cell_type<-celltype

In [ ]:
celltype<-vector()
for (i in 1:dim(LS_nosal@meta.data)[1]){
    celltype[i]<-toString(new.ident[LS_nosal@meta.data$integrated_snn_res.0.2[i]])
}
LS_nosal@meta.data$cell_type<-celltype

In [ ]:
celltype<-vector()
for (i in 1:dim(LS.integrated@meta.data)[1]){
    celltype[i]<-toString(new.ident[LS.integrated@meta.data$integrated_snn_res.0.2[i]])
}
LS.integrated@meta.data$cell_type<-celltype

In [ ]:
augur_nonal = calculate_auc(LS_nonal, n_threads = 1, label_col="stim", cell_type_col = "cell_type")

In [ ]:
augur_nosal = calculate_auc(LS_nosal, n_threads = 1, label_col="stim", cell_type_col = "cell_type")

In [ ]:
augur_nosal$AUC

In [ ]:
augur_nonal$AUC

Saving .rds files

In [ ]:
saveRDS(augur_nonal, file='augur_sal_v_mor.rds')

In [ ]:
saveRDS(augur_nosal, file='augur_mor_v_nal.rds')

#### Generating a shuffled dataset for comparisons

In [ ]:
set.seed(5)

In [ ]:
LS_sal_shuffle <- rownames(LS.integrated@meta.data[LS.integrated@meta.data$stim == "LS_sal",])
LS_mor_shuffle <- rownames(LS.integrated@meta.data[LS.integrated@meta.data$stim == "LS_mor",])
LS_nal_shuffle <- rownames(LS.integrated@meta.data[LS.integrated@meta.data$stim == "LS_nal",])

In [ ]:
LS_nosal_shuffle <- subset(LS.integrated, cells=c(LS_mor_shuffle, LS_nal_shuffle))
LS_nosal_shuffle

In [ ]:
LS_nonal_shuffle <- subset(LS.integrated, cells=c(LS_sal_shuffle, LS_mor_shuffle))
LS_nonal_shuffle

In [ ]:
saveRDS(augur_nonal_shuffle, file='augur_sal_v_mor_shuffle.rds')

In [ ]:
saveRDS(augur_nosal_shuffle, file='augur_mor_v_nal_shuffle.rds')